# 컬럼 조정으로 f1 score 향상되는지 확인

In [7]:
import pandas as pd
df = pd.read_csv('../data/raw_postpro.csv')

In [17]:
df.columns
# df.info()

Index(['청구서번호', 'No.', 'Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.1',
       'Part No.2', 'key1', 'key2', '리드타임_음수제거', '청구량', '견적', '견적수량', '견적화폐',
       '견적단가', '발주번호', '발주처', '발주', '발주수량', '발주금액', 'D/T', '미입고 기간', '리드타임',
       '창고입고', '창고입고수량', 'Control No.', '입고창고', '창고출고', '창고출고수량', '출고선박',
       '출고운반선', '선박입고', '선박입고수량', '완료 여부', 'leadtime'],
      dtype='object')

# 분류에 필요없다고 판단되는 컬럼 드랍

In [18]:
df = df.drop(['청구서번호','No.',  '선박입고','완료 여부','리드타임_음수제거','청구량','견적','견적수량','견적화폐','견적단가','발주번호','발주','발주수량','발주금액','미입고 기간','리드타임','창고입고','창고입고수량','입고창고','창고출고','창고출고수량','출고선박','출고운반선','선박입고','선박입고수량','완료 여부','leadtime'], axis=1)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Subject      20517 non-null  object
 1   Machinery    20517 non-null  object
 2   Assembly     20517 non-null  object
 3   청구품목         20517 non-null  object
 4   Part No.1    20517 non-null  object
 5   Part No.2    20517 non-null  object
 6   key1         20517 non-null  object
 7   key2         20517 non-null  object
 8   발주처          20517 non-null  object
 9   D/T          20517 non-null  object
 10  Control No.  20517 non-null  object
dtypes: object(11)
memory usage: 1.7+ MB


In [29]:
import itertools

# 데이터프레임의 열 목록
columns = df.columns.tolist()

# 5개의 열로 가능한 모든 조합 생성
combinations = list(itertools.combinations(columns, 5))

# 조합 개수 출력
print(len(combinations))

462


In [30]:
combinations = [list(combo) for combo in combinations]

In [53]:
# combinations

In [31]:
y = df["key2"]
le_y = preprocessing.LabelEncoder()
y = le_y.fit_transform(y)

In [57]:
def making_X(combo):
    le_x = preprocessing.LabelEncoder()
    X = df[combo]
    X = X.apply(le_x.fit_transform)
    return X

In [50]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

In [62]:
def train_model(combo, X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=61)

    # 모델 학습
    model.fit(X_train, y_train)
    # 예측 결과 출력
    y_pred = model.predict(X_test)
    # 리턴 값
    accu_score =  accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accu_score,f1

In [64]:
# combo = ['Machinery', 'Part No.1', 'Part No.2', 'D/T', 'Control No.']

In [ ]:
resultset = {}
max_accu = 0
max_f1=0
for combo in combinations:
    X = making_X(combo)
    a, f1 = train_model(combo, X,y)
    if a > max_accu:
        max_accu = a
        resultset['max_accu'] = max_accu
        resultset['max_accu_combo'] = combo
    if f1 > max_f1:
        max_f1 = f1
        resultset['max_f1'] = max_accu
        resultset['max_f1_combo'] = combo
    print('combo : ', combo)
    print('accuracy_score : ' , a)
    print('f1_score : ' , f1) 
    print('---------------------------')

In [63]:
# X = making_X(combo)
# a, f1 = train_model(combo, X,y)
# print('accuracy_score : ' , a)
# print('f1_score : ' , f1) 
# print('---------------------------')

accuracy_score :  0.7331871345029239
f1_score :  0.7304564872165126
---------------------------


In [66]:
# resultset = {}
# max_accu = 0
# max_f1=0
# X = making_X(combo)
# a, f1 = train_model(combo, X,y)
# if a > max_accu:
#     max_accu = a
#     resultset['max_accu'] = max_accu
#     resultset['max_accu_combo'] = combo
# if f1 > max_f1:
#     max_f1 = f1
#     resultset['max_f1'] = max_accu
#     resultset['max_f1_combo'] = combo
# print('combo : ', combo)
# print('accuracy_score : ' , a)
# print('f1_score : ' , f1) 
# print('---------------------------')

combo :  ['Machinery', 'Part No.1', 'Part No.2', 'D/T', 'Control No.']
accuracy_score :  0.7331871345029239
f1_score :  0.7304564872165126
---------------------------


In [67]:
resultset

{'max_accu': 0.7331871345029239,
 'max_accu_combo': ['Machinery',
  'Part No.1',
  'Part No.2',
  'D/T',
  'Control No.'],
 'max_f1': 0.7331871345029239,
 'max_f1_combo': ['Machinery', 'Part No.1', 'Part No.2', 'D/T', 'Control No.']}